In [4]:
import duckdb
import pandas as pd

In [5]:
# Connecting to DB
con = duckdb.connect("../github_stars.db")

In [9]:
query = """
    SELECT *
    FROM github_stars.main_marts.fact_stars_monthly as s
    JOIN github_stars.main_marts.dim_repos as r
    ON s.repo_id = r.repo_id
    AND end_date IS NULL
"""

df_monthly_stars = con.sql(query).df()
df_monthly_stars.to_csv("monthly_stars.csv")

CatalogException: Catalog Error: Table with name fact_stars_monthly does not exist!
Did you mean "github_stars.fact_stars_monthly"?